## License Plate Detection using Fuzzy Join 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Connect to EvaDB

In [2]:
%pip install evadb
import evadb
cursor = evadb.connect().cursor()

Note: you may need to restart the kernel to use updated packages.


### Loading the images to EvaDB for analysis

In [6]:
# Download images
!wget -nc "https://www.dropbox.com/s/770stddqfl0psog/license.zip"
!unzip -n license.zip

cursor.query('DROP TABLE IF EXISTS MyImages;').df()

cursor.load("license/Car*.png", "MyImages", format="image").df()

File 'license.zip' already there; not retrieving.

Archive:  license.zip


06-06-2023 20:53:26 WARNING[drop_executor:drop_executor.py:exec:0045] Table: MyImages does not exist


,0
0,Number of loaded IMAGE: 7


### License Plate Recognition

In [7]:
# Download UDFS

!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/evadb/udfs/ndarray/fuzzy_join.py

!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/evadb/udfs/ocr_extractor.py

cursor.query("""DROP UDF IF EXISTS OCRExtractor;""").df()

cursor.query("""DROP UDF IF EXISTS FuzzDistance;""").df()

cursor.create_udf("OCRExtractor", True, 'ocr_extractor.py').df()
cursor.create_udf("FuzzDistance", True, 'fuzzy_join.py').df()

File 'fuzzy_join.py' already there; not retrieving.

File 'ocr_extractor.py' already there; not retrieving.



06-06-2023 20:53:31 WARNING[drop_udf_executor:drop_udf_executor.py:exec:0039] UDF OCRExtractor does not exist, therefore cannot be dropped.
06-06-2023 20:53:31 WARNING[drop_udf_executor:drop_udf_executor.py:exec:0039] UDF FuzzDistance does not exist, therefore cannot be dropped.
06-06-2023 20:53:35 ERROR [plan_executor:plan_executor.py:execute_plan:0169] Cannot infer io signature from the decorator for <class 'ocr_extractor.OCRExtractor'>.
Traceback (most recent call last):
  File "/nethome/jarulraj3/eva/test_evadb/lib/python3.10/site-packages/eva/executor/plan_executor.py", line 167, in execute_plan
    yield from output
  File "/nethome/jarulraj3/eva/test_evadb/lib/python3.10/site-packages/eva/executor/create_udf_executor.py", line 114, in exec
    name, impl_path, udf_type, io_list, metadata = self.handle_generic_udf()
  File "/nethome/jarulraj3/eva/test_evadb/lib/python3.10/site-packages/eva/executor/create_udf_executor.py", line 82, in handle_generic_udf
    io_list = self._resolv

ExecutorError: Cannot infer io signature from the decorator for <class 'ocr_extractor.OCRExtractor'>.

In [ ]:
cursor.query(
    "CREATE TABLE IF NOT EXISTS LicensePlateCSV(id INTEGER UNIQUE, label TEXT(30));"
).df()

""


In [ ]:
cursor.load("data.csv", "LicensePlateCSV").df()

NameError: name 'cursor' is not defined

In [ ]:
query = cursor.connect("MyImages")
query = query.select("OCRExtractor(data)")

query.df()

,ocrextractor.labels,ocrextractor.bboxes,ocrextractor.scores
0,[KLo1CN2555],"[[[240, 115], [425, 115], [425, 178], [240, 17...",[0.20029704378116528]
1,"[Coi, ZZm, PGoNMN112]","[[[320, 96], [342, 96], [342, 104], [320, 104]...","[0.2585151841538518, 0.009615490492723363, 0.1..."
2,[PReINLR],"[[[227, 173], [269, 173], [269, 193], [227, 19...",[0.12317288475094909]
3,[DZI7 YXR],"[[[150, 126], [250, 126], [250, 158], [150, 15...",[0.6978633091234704]
4,"[pPuie, BES]","[[[175.83355283634924, 85.05243648347647], [38...","[0.19272780155017014, 0.9999165839414489]"
5,"[Wet, alamy stock photo]","[[[362, 274], [384, 274], [384, 280], [362, 28...","[0.002928912305910474, 0.9390591079980826]"
6,"[802.LIN, MMay, VIRGINIA, 07]","[[[157, 141], [370, 141], [370, 219], [157, 21...","[0.8629838375738593, 0.3385108709335327, 0.999..."


In [ ]:
query = cursor.connect("LicensePlateCSV")
query = query.select("*")

query.df()

,licenseplatecsv._row_id,licenseplatecsv.id,licenseplatecsv.label
0,1,1,KLG1CA2555
1,2,2,PGMN112
2,3,3,PRENUP
3,4,4,DZ17YXR
4,5,5,PUI8BES


### Run Fuzzy Join to match Detected License Plate against Local License Plate Database (csv)

In [ ]:
cursor.query("""
   SELECT * FROM MyImages 
       JOIN LATERAL OCRExtractor(data) AS T(a,b,c) 
       JOIN LicensePlateCSV B 
       ON FuzzDistance(T.a, B.label) > 50;
       """).df()

,myimages._row_id,myimages.name,myimages.data,B._row_id,B.id,B.label,T.a,T.b,T.c
0,1,license/Cars0.png,"[[[25, 75, 100], [73, 130, 159], [52, 127, 158...",1,1,KLG1CA2555,[KLo1CN2555],"[[[240, 115], [425, 115], [425, 178], [240, 17...",[0.20029704378116528]
